# Chassis.ml demo

## Easily build MLflow models into {KFServing, Modzy} Docker images

This demo will show you how we can train a model, define custom pre- and post-processing steps, save it in MLflow format and then build it into a container image and push it to docker hub with a single command.

By easily connecting MLflow models to Docker images with a simple Python SDK for data scientists & ML engineers, Chassis is the missing link between MLflow and DevOps.

This demo can be run in local using minikube and a local installation of Chassis.

## Prerequisites

* [Docker Hub](https://hub.docker.com/) account (free one is fine)
* The browser you're reading this in :-)
* Existing local installation of Chassis

In [1]:
import chassisml
import sklearn
import mlflow.pyfunc
from joblib import dump, load

### Train the model

This will train a sklearn model and it will be saved as a joblib file inside the `model` directory.

The goal for Chassis service is to create an image that exposes this model.

In [2]:
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)
dump(clf, './model.joblib')

['./model.joblib']

In [7]:
# Wrap your model in a pyfunc and provide auxiliary functionality through extension of the
# mlflow PythonModel class with methods pre_process, post_process, and explain

class CustomModel(mlflow.pyfunc.PythonModel):
    _model = load('./model.joblib')
    
    def load_context(self, context):
        self.model = self._model

    def predict(self, context, input_dict):
        processed_inputs = self.pre_process(input_dict['input_data_bytes'])
        inference_results = self.model.predict(processed_inputs)
        return self.post_process(inference_results)

    def pre_process(self, input_bytes):
        import json
        import numpy as np
        inputs = np.array(json.loads(input_bytes))
        return inputs / 2

    def post_process(self, inference_results):
        structured_results = []
        for inference_result in inference_results:
            inference_result = {
                "classPredictions": [
                    {"class": str(inference_result), "score": str(1)}
                ]
            }
            structured_output = {
                "data": {
                    "result": inference_result,
                    "explanation": None,
                    "drift": None,
                }
            }
            structured_results.append(structured_output)
        return structured_results

    def explain(self, images):
        pass

In [8]:
# Define conda environment with all required dependencies for your model

conda_env = {
    "channels": ["defaults", "conda-forge", "pytorch"],
    "dependencies": [
        "python=3.8.5",
        "pip",
        {
            "pip": [
                "mlflow",
                "sklearn"
            ],
        },
    ],
    "name": "linear_env"
}

### Save the model

Transform the model into MLFlow format.

In [28]:
!rm -rf mlflow_custom_pyfunc_svm
model_save_path = "mlflow_custom_pyfunc_svm"
mlflow.pyfunc.save_model(path=model_save_path, python_model=CustomModel(), conda_env=conda_env)

Load the MLFlow model and test it.

In [10]:
import json

input_dict = {'input_data_bytes': open('./modzy/input_sample.json','rb').read()}

classifier = mlflow.pyfunc.load_model(model_save_path)
predictions = classifier.predict(input_dict)
print(json.dumps(predictions, indent=4))

[
    {
        "data": {
            "result": {
                "classPredictions": [
                    {
                        "class": "4",
                        "score": "1"
                    }
                ]
            },
            "explanation": null,
            "drift": null
        }
    },
    {
        "data": {
            "result": {
                "classPredictions": [
                    {
                        "class": "8",
                        "score": "1"
                    }
                ]
            },
            "explanation": null,
            "drift": null
        }
    },
    {
        "data": {
            "result": {
                "classPredictions": [
                    {
                        "class": "8",
                        "score": "1"
                    }
                ]
            },
            "explanation": null,
            "drift": null
        }
    },
    {
        "data": {
            "result": {
        

We check that the model has been correctly saved inside the `model` directory.

In [11]:
!ls ./mlflow_custom_pyfunc_svm

MLmodel  conda.yaml  python_model.pkl  requirements.txt


### Get Docker Hub credentials securely

Now we prompt the user (you!) for your docker hub username and password in such a way that the value itself doesn't get written into the notebook, which is sensible security best-practice.

In [12]:
import getpass
import base64
username = getpass.getpass('docker hub username')
password = getpass.getpass('docker hub password')

docker hub username········
docker hub password········


### Complete the data

Now we can construct the metadata that the chassis service needs to build and publish the container to docker hub. In case `publish` is `False` the image is not uploaded to Docker Hub.

In [13]:
image_data = {
    'name': f'{username}/chassisml-sklearn-digits:latest',
    'version': '0.0.1',
    'model_name': 'digits',
    'model_path': './mlflow_custom_pyfunc_svm',
    'registry_auth': base64.b64encode(f"{username}:{password}".encode("utf-8")).decode("utf-8"),
    'publish': True
}

#### Complete Modzy data

In the case of KFServing, no more data is required. When it comes to Modzy, we will need to define some more data:

* `metadata_path`: this is the path to the [model.yaml](https://models.modzy.com/docs/model-packaging/model-packaging-python-template/yaml-file_) file that is needed to define all information about the model. Chassis has a default one, but you should define your own based on [this example](https://github.com/modzy/chassis/blob/main/chassisml-sdk/examples/modzy/model.yaml)
* `sample_input_path`: this is the path to the [sample input](https://models.modzy.com/docs/model-deployment/model-deployment/input-outputs) that is needed when deploying the model. An example can be found [here](https://github.com/modzy/chassis/blob/main/chassisml-sdk/examples/modzy/input_sample.json)
* `deploy`: if it is `True` Chassis will manage to deploy the model into Modzy platform. Otherwise you can do this manually through the Modzy UI
* `api_key`: you should have your own [api key](https://models.modzy.com/docs/how-to-guides/api-keys) from Modzy in order to let Chassis deploy the model for you

Notice that if `deploy` is False this means that you can avoid defining the rest of the fields. Anyway, `metadata_path` should be defined in case you will eventually deploy the model to Modzy. This is important because the model will use this information when being deployed to Modzy, so it needs to be updated.

In [14]:
import getpass
modzy_api_key = getpass.getpass('modzy api_key')

modzy api_key········


In [15]:
modzy_data = {
    'metadata_path': './modzy/model.yaml',
    'sample_input_path': './modzy/input_sample.json',
    'deploy': True,
    'api_key': modzy_api_key
}

### Forward ports to access service and registry

This assumes that you are running these commands on your own terminal to redirect the service (port 5000) and the registry (port 5001) to localhost.

In [16]:
! # kubectl port-forward service/chassis 5000:5000

### Launch the job

Important fields that we should fill in here are:

* `module`: library that has been used to create the model
* `image_data`: the values defined above
* `image_type`: this is needed in case we are training images so afterwards the proxy will know how to interpret data
* `base_url`: the name of the service that runs Chassis

In [17]:
res = chassisml.publish(
    image_data=image_data,
    modzy_data=modzy_data,
    base_url='http://localhost:5000'
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

Building image... Ok!
Job ID: chassis-builder-job-7bf9052c-b0ff-4d3b-bbd2-1bd15ff07f18


After the request is made, Chassis launches a job that runs Kaniko and builds the docker image based on the values provided.

You can get the id of the job created from the result of the request. This id can be used to ask for the status of the job.

This is an example of the data that is shown when the job has not finished yet.

In [23]:
chassisml.get_job_status(job_id)

{'result': {'containerImage': {'containerImageSize': 0,
   'loadPercentage': 10,
   'loadStatus': 'IN_PROGRESS',
   'repositoryName': 'uzo5lhkph1',
   'uploadPercentage': 0,
   'uploadStatus': 'IN_PROGRESS'},
  'container_url': 'https://integration.modzy.engineering/models/uzo5lhkph1/0.1.0',
  'createdAt': '2021-11-09T23:30:59.714+00:00',
  'inputValidationSchema': '',
  'inputs': [{'acceptedMediaTypes': 'application/json',
    'description': 'numpy array 2d',
    'maximumSize': 1000000,
    'name': 'input.json'}],
  'isActive': False,
  'isAvailable': True,
  'longDescription': 'It classifies digits.',
  'model': {'author': 'Integration',
   'createdByEmail': 'saumil.dave@modzy.com',
   'description': 'This is an image built by chassis that exposes a sklearn model.',
   'features': [],
   'isActive': False,
   'isCommercial': False,
   'isRecommended': False,
   'latestActiveVersion': '',
   'latestVersion': '0.1.0',
   'modelId': 'uzo5lhkph1',
   'name': 'chassis-digits-test',
   'pe

And this is an example of the data that is shown when the job has already finished.

Two top keys can be seen:

* `result`: in case `deploy` was `True` this will contain some information related to the model deployed in Modzy. In particular we can see the full url to the model if we access the `container_url` key.
* `status`: this contains the information about the kubernetes job that has built the image (and uploaded it to Modzy in case `deploy` was `True` as explained above)

In [24]:
job_status = chassisml.get_job_status(job_id)
result = job_status.get('result')

job_status

{'result': {'containerImage': {'containerImageSize': 0,
   'loadPercentage': 10,
   'loadStatus': 'IN_PROGRESS',
   'repositoryName': 'uzo5lhkph1',
   'uploadPercentage': 0,
   'uploadStatus': 'IN_PROGRESS'},
  'container_url': 'https://integration.modzy.engineering/models/uzo5lhkph1/0.1.0',
  'createdAt': '2021-11-09T23:30:59.714+00:00',
  'inputValidationSchema': '',
  'inputs': [{'acceptedMediaTypes': 'application/json',
    'description': 'numpy array 2d',
    'maximumSize': 1000000,
    'name': 'input.json'}],
  'isActive': False,
  'isAvailable': True,
  'longDescription': 'It classifies digits.',
  'model': {'author': 'Integration',
   'createdByEmail': 'saumil.dave@modzy.com',
   'description': 'This is an image built by chassis that exposes a sklearn model.',
   'features': [],
   'isActive': False,
   'isCommercial': False,
   'isRecommended': False,
   'latestActiveVersion': '',
   'latestVersion': '0.1.0',
   'modelId': 'uzo5lhkph1',
   'name': 'chassis-digits-test',
   'pe

### Inference!

Now that the model has been deployed into Modzy platform, we can make a request against it to see it working. 

This is going to use the sample input defined above, which is going to be wrapped in a [Modzy Job](https://models.modzy.com/docs/jobs/jobs/submit-job-text). Take into account that input names must match model input filenames.

In [25]:
input_name = result['inputs'][0]['name'] # e.g. input.json

request_data = {
  'model': {
    'identifier': f'{result.get("model").get("modelId")}',
    'version': f'{result.get("version")}'
  },
  'input': {
    'type': 'text',
    'sources': {
      'input': {
        input_name: '[[0.0, 0.0, 0.0, 1.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 6.0, 1.0, 2.0, 0.0, 0.0, 4.0, 16.0, 9.0, 1.0, 15.0, 9.0, 0.0, 0.0, 13.0, 15.0, 6.0, 10.0, 16.0, 6.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 1.0, 7.0, 4.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 9.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 16.0, 14.0, 5.0, 14.0, 12.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 10.0, 1.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 8.0, 0.0, 1.0, 0.0, 0.0, 0.0, 11.0, 16.0, 5.0, 13.0, 6.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 3.0, 4.0, 3.0, 0.0, 0.0, 7.0, 16.0, 5.0, 3.0, 15.0, 8.0, 0.0, 0.0, 13.0, 16.0, 13.0, 15.0, 16.0, 2.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0], [0.0, 0.0, 10.0, 14.0, 8.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 15.0, 12.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 4.0, 16.0, 6.0, 0.0, 0.0, 8.0, 16.0, 10.0, 8.0, 16.0, 8.0, 0.0, 0.0, 1.0, 8.0, 12.0, 14.0, 12.0, 1.0, 0.0]]'
        # ^ this has to match the input filename specified when the model was created
      }
    }
  }
}

import requests

res_job = requests.post(
    'https://integration.modzy.engineering/api/jobs',
    json=request_data,
    headers={'Authorization': f'ApiKey {modzy_api_key}'}
)

res_job_json = res_job.json()

res_job_json

{'model': {'identifier': 'uzo5lhkph1',
  'version': '0.1.0',
  'name': 'chassis-digits-test'},
 'status': 'SUBMITTED',
 'totalInputs': 1,
 'jobIdentifier': '1f840b3d-60a0-4c72-90da-de34523c26e1',
 'accessKey': 'q6ED7pBAFtCsDkbaBLpt',
 'explain': False,
 'jobType': 'batch',
 'accountIdentifier': 'modzy-account',
 'team': {'identifier': '830b2012-1557-48a9-a4df-d867b5a0939a'},
 'user': {'identifier': 'eb8cf7ea-3930-410d-812f-7430e6baf3c5',
  'externalIdentifier': 'saumil.dave@modzy.com',
  'firstName': 'Saumil',
  'lastName': 'Dave',
  'email': 'saumil.dave@modzy.com',
  'status': 'active'},
 'jobInputs': {'identifier': ['input']},
 'submittedAt': '2021-11-09T23:32:33.121+00:00',
 'hoursDeleteInput': 24,
 'imageClassificationModel': False}

We must wait until the job has finished and we can see the results.

In [27]:
job_id = res_job_json.get('jobIdentifier')

res_result = requests.get(
    f'https://integration.modzy.engineering/api/results/{job_id}',
    headers={'Authorization': f'ApiKey {modzy_api_key}'}
)

res_result_json = res_result.json()

res_result_json

{'jobIdentifier': '1f840b3d-60a0-4c72-90da-de34523c26e1',
 'accountIdentifier': 'modzy-account',
 'team': {'identifier': '830b2012-1557-48a9-a4df-d867b5a0939a'},
 'total': 1,
 'completed': 1,
 'failed': 0,
 'finished': True,
 'submittedByKey': 'q6ED7pBAFtCsDkbaBLpt',
 'explained': False,
 'submittedAt': '2021-11-09T23:32:33.121+00:00',
 'initialQueueTime': 120733,
 'totalQueueTime': 120733,
 'averageModelLatency': 2799.0,
 'totalModelLatency': 2799.0,
 'elapsedTime': 124879,
 'startingResultSummarizing': '2021-11-09T23:34:37.304+00:00',
 'resultSummarizing': 696,
 'inputSize': 506,
 'results': {'input': {'status': 'SUCCESSFUL',
   'engine': 'model-batch-uzo5lhkph1-0-1-0-b8dbf6c8f-vfpxv',
   'inputFetching': 2593,
   'outputUploading': None,
   'modelLatency': 2799.0,
   'queueTime': 120733,
   'startTime': '2021-11-09T23:34:33.647+0000',
   'updateTime': '2021-11-09T23:34:37.035+0000',
   'endTime': '2021-11-09T23:34:37.035+0000',
   'results.json': [{'data': {'result': {'classPredicti